# Database programming: SQLite

SQLite is an embedded database engine (there is no separate server) that is included with Python.

In [14]:
import sqlite3
conn = sqlite3.connect('data/real-estate.db')

In [19]:
sqlite3.connect?

Docstring:
connect(database[, timeout, detect_types, isolation_level,
        check_same_thread, factory, cached_statements, uri])

Opens a connection to the SQLite database file *database*. You can use
":memory:" to open a database connection to a database that resides in
RAM instead of on disk.
Type:      builtin_function_or_method


In [13]:
cursor = conn.execute('SELECT * FROM "transactions" LIMIT 1')

In [15]:
cursor.description

(('index', None, None, None, None, None, None),
 ('street', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('zip', None, None, None, None, None, None),
 ('state', None, None, None, None, None, None),
 ('beds', None, None, None, None, None, None),
 ('baths', None, None, None, None, None, None),
 ('sq__ft', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None),
 ('sale_date', None, None, None, None, None, None),
 ('price', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None))

In [16]:
for row in cursor:
    print(row)

(0, '3526 HIGH ST', 'SACRAMENTO', 95838, 'CA', 2, 1, 836, 'Residential', 'Wed May 21 00:00:00 EDT 2008', 59222, 38.631913, -121.43487900000001)


In [17]:
for row in conn.execute('SELECT type, count(*) from transactions GROUP BY type'):
    print(row)

('Condo', 54)
('Multi-Family', 13)
('Residential', 917)
('Unkown', 1)


In [6]:
conn.execute("select * from transactions where type = 'Unkown'").fetchone()

(757,
 '6007 MARYBELLE LN',
 'SHINGLE SPRINGS',
 95682,
 'CA',
 0,
 0,
 0,
 'Unkown',
 'Fri May 16 00:00:00 EDT 2008',
 275000,
 38.64347,
 -120.88818300000001)

# Please don't do this
```python
user_provided_data = "SACRAMENTO'; DROP TABLE transactions; -- "
format_str = "SELECT * FROM transactions WHERE city = '%s'"" % (user_provided_data)
```

https://xkcd.com/327/

# Do this instead

"Bind" parameters:

In [7]:
stmt = 'SELECT count(*), avg(price) FROM transactions WHERE type = ?'

In [17]:
cursor = conn.execute(stmt, ('Condo',))
for row in cursor:
    print(row)

(54, 150082.1851851852)


In [8]:
for row in conn.execute(stmt, ('Condo',)):
    print(row)

(54, 150082.1851851852)


In [9]:
for row in conn.execute(stmt, ('Residential\'); --)',)):
    print(row)

(0, None)


Writing data

In [10]:
c = conn.cursor()
c.execute("""CREATE TABLE stocks(
    date text, 
    symbol text, 
    price real
)""")

In [11]:
data = [
    ("2014-01-02", "F", 12.089),
    ("2014-01-02", "TSLA", 150.1),
    ("2014-01-02", "IBM", 157.6001),
    ("2014-01-02", "AAPL", 72.7741),
    ("2014-01-03", "F", 12.1438),
    ("2014-01-03", "TSLA", 149.56),
    ("2014-01-03", "IBM", 158.543),
    ("2014-01-03", "AAPL", 71.1756),
    ("2014-01-06", "F", 12.1986),
    ("2014-01-06", "TSLA", 147.0),
    ("2014-01-06", "IBM", 157.9993),
    ("2014-01-06", "AAPL", 71.5637),
    ("2014-01-07", "F", 12.042),
    ("2014-01-07", "TSLA", 149.36),
    ("2014-01-07", "IBM", 161.1508),
    ("2014-01-07", "AAPL", 71.0516),
    ("2014-01-08", "F", 12.1673),
    ("2014-01-08", "TSLA", 151.28),
    ("2014-01-08", "IBM", 159.6728),
    ("2014-01-08", "AAPL", 71.5019),
]
data = iter(data)

In [12]:
c.executemany('INSERT INTO stocks VALUES(?, ?, ?)', data)

In [13]:
conn.commit()

In [14]:
conn.execute("SELECT * FROM stocks WHERE symbol = 'F'").fetchall()

[('2014-01-02', 'F', 12.089),
 ('2014-01-03', 'F', 12.1438),
 ('2014-01-06', 'F', 12.1986),
 ('2014-01-07', 'F', 12.042),
 ('2014-01-08', 'F', 12.1673)]

In [18]:
with conn:
    conn.execute("DELETE FROM stocks WHERE symbol = 'TSLA'")

In [19]:
c.execute("DROP TABLE stocks")

# Database programming: MySQL

To use MySQL, we'll need to install the pymysql driver:

In [29]:
!pip3 install pymysql

  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [32]:
%%bash
pip install mysql
mysql

Error processing line 1 of /Users/sridharmundra/Library/Python/3.8/lib/python/site-packages/pyston_autoload.pth:

  Traceback (most recent call last):
    File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<string>", line 1, in <module>
  ImportError: dlopen(/Users/sridharmundra/Library/Python/3.8/lib/python/site-packages/pyston.cpython-38-darwin.so, 0x0002): symbol not found in flat namespace (__PyAsyncGenValueWrapperNew)

Remainder of file ignored


Defaulting to user installation because normal site-packages is not writeable
  Using cached mysql-0.0.3-py3-none-any.whl (1.2 kB)
  Using cached mysqlclient-2.1.1.tar.gz (88 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [26 lines of output]
      /bin/sh: mysql_config: command not found
      /bin/sh: mariadb_config: command not found
      /bin/sh: mysql_config: command not found
      Error processing line 1 of /Users/sridharmundra/Library/Python/3.8/lib/python/site-packages/pyston_autoload.pth:
      
        Traceback (most recent call last):
          File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.8/lib/python3.8/site.py", line 169, in addpackage
            exec(line)
          File "<string>", line 1, in <module>
          File "<string>", line 1, in <module>
        ImportError: dlopen(/Users/sridharmundra/Library/Python/3.8/lib/python/site-packages/pyston.cpython-38-darwin.so, 0x0002): symbol not found in flat namespace (__PyAsyncGenValueWrapperNew)
      
      Remainder of file ignored
      Traceback (most recent call last):
     

CalledProcessError: Command 'b'pip install mysql\nmysql\n'' returned non-zero exit status 127.

In [36]:
import pymysql

host = 'localhost'
conn = pymysql.connect(
    host=host,
    user='root',
    password='password',
    db='class',
)

OperationalError: (2003, "Can't connect to MySQL server on 'localhost' ([Errno 61] Connection refused)")

In [37]:
with conn.cursor() as c:
    print(c, type(c))
    value = c.execute('SELECT type, count(*) from transactions GROUP BY type')
    print(c.description)
    print(value)
    for item in c:
        print(item)

AttributeError: __enter__

In [23]:
conn

In [24]:
with conn.cursor() as c:
    c.execute("""CREATE TABLE stocks(
        date text, 
        symbol text, 
        price real
    )""")

In [25]:
with sqlite3.connect('./data/stocks.db') as sqlite_conn:
    data = list(sqlite_conn.execute('SELECT date, symbol, price FROM stocks_data'))

In [26]:
data[:1]

[('2014-01-02', 'F', 12.089)]

In [27]:
sqlite_conn = sqlite3.connect('./data/stocks.db')

with conn.cursor() as mysql_cursor:
    sqlite_cursor = sqlite_conn.cursor()
    data = sqlite_cursor.execute('SELECT date, symbol, price FROM stocks_data')
    mysql_cursor.execute('DELETE FROM stocks;')
    mysql_cursor.executemany('INSERT INTO stocks VALUES(%s, %s, %s)', data)

In [28]:
with conn.cursor() as c:
    q = c.execute("SELECT symbol, count(*) FROM stocks group by symbol")
    print(list(c))


[('F', 1007), ('TSLA', 1007), ('GOOG', 1007), ('IBM', 1007), ('AAPL', 1007)]


In [29]:
with conn.cursor() as c:
    q = c.execute("SELECT * FROM stocks WHERE symbol = 'IBM' LIMIT 5")
    print(c.description)
    for row in c:
        print(row)


(('date', 252, None, 262140, 262140, 0, True), ('symbol', 252, None, 262140, 262140, 0, True), ('price', 5, None, 22, 22, 31, True))
('2014-01-02', 'IBM', 157.6001)
('2014-01-03', 'IBM', 158.543)
('2014-01-06', 'IBM', 157.9993)
('2014-01-07', 'IBM', 161.1508)
('2014-01-08', 'IBM', 159.6728)


In [30]:
with conn.cursor() as c:
    c.execute("DELETE FROM stocks WHERE symbol = 'TSLA'")

In [31]:
with conn.cursor() as c:
    c.execute("DROP TABLE stocks")

Open [DBAPI lab](./dbapi-lab.ipynb)